In [1]:
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q https://downloads.apache.org/spark/spark-3.1.1/spark-3.1.1-bin-hadoop3.2.tgz
!tar xf spark-3.1.1-bin-hadoop3.2.tgz
!pip install -q findspark

In [2]:
# retart after inatall
!pip install pyspark

     |████████████████████████████████| 212.3MB 66kB/s 
     |████████████████████████████████| 204kB 36.1MB/s 
  Created wheel for pyspark: filename=pyspark-3.1.1-py2.py3-none-any.whl size=212767604 sha256=c7d7d8a0422275e0e82e60df113fa89891e55629e4a1922a216b278d9de7f8fb
  Stored in directory: /root/.cache/pip/wheels/0b/90/c0/01de724414ef122bd05f056541fb6a0ecf47c7ca655f8b3c0f
Successfully built pyspark


# SparkContxt 생성

SparkContext
- Main entry point for spark functionality
- A SparkContext represents the connection to a spark cluster, and can used to create RDDs, accumulators and broadcast variables on that cluster.
- Only one SparkContext may be active per JVM.
- You must stop() the active SparkContext before creating a new one.

In [3]:
from pyspark import SparkContext, SparkConf

**First way to create SparkContext**

In [4]:
# 스파크 설정
config = SparkConf().setAppName('Demo').setMaster('local')

# 스파크 문맥 생성
sc = SparkContext(conf=config)

In [5]:
# 스파크 문맥 설정 보기
sc.getConf().getAll()

[('spark.master', 'local'),
 ('spark.app.id', 'local-1620565123905'),
 ('spark.driver.port', '38337'),
 ('spark.rdd.compress', 'True'),
 ('spark.serializer.objectStreamReset', '100'),
 ('spark.submit.pyFiles', ''),
 ('spark.executor.id', 'driver'),
 ('spark.app.startTime', '1620565122267'),
 ('spark.submit.deployMode', 'client'),
 ('spark.driver.host', '92bef1ff7a89'),
 ('spark.ui.showConsoleProgress', 'true'),
 ('spark.app.name', 'Demo')]

In [6]:
sc.stop()

**Second way, Create a SparkContext that loads settings**

In [7]:
sc = SparkContext()

In [8]:
sc.getConf().getAll()

[('spark.master', 'local'),
 ('spark.app.id', 'local-1620565125439'),
 ('spark.app.startTime', '1620565125271'),
 ('spark.driver.port', '46155'),
 ('spark.rdd.compress', 'True'),
 ('spark.serializer.objectStreamReset', '100'),
 ('spark.submit.pyFiles', ''),
 ('spark.executor.id', 'driver'),
 ('spark.submit.deployMode', 'client'),
 ('spark.driver.host', '92bef1ff7a89'),
 ('spark.ui.showConsoleProgress', 'true'),
 ('spark.app.name', 'Demo')]

In [9]:
sc.stop()

**Third way to combining both**

In [10]:
sc = SparkContext('local','First App')

In [11]:
sc.getConf().getAll()

[('spark.master', 'local'),
 ('spark.rdd.compress', 'True'),
 ('spark.app.startTime', '1620565126034'),
 ('spark.driver.port', '33505'),
 ('spark.serializer.objectStreamReset', '100'),
 ('spark.submit.pyFiles', ''),
 ('spark.executor.id', 'driver'),
 ('spark.submit.deployMode', 'client'),
 ('spark.app.name', 'First App'),
 ('spark.driver.host', '92bef1ff7a89'),
 ('spark.ui.showConsoleProgress', 'true'),
 ('spark.app.id', 'local-1620565126183')]

**RDD**
- Transfomation : 새로운 RDD를 생성하기 위한 DAG 생성한다.
    - narrow transformation과 wide transformation으로 나뉜다.
- Action : Action을 실행하면 Transformation이 실행되면 RDD가 생성되며(Lazy execution) 값을 반환한다.

# RDD 생성과 기초 Action들

In [12]:
names = sc.parallelize(['Adam','Cray','Shaun','Brain','Mark','Christ','Shail','Satya','Mark','Norby','Frans','Mark','Bill'])

In [13]:
type(names)

pyspark.rdd.RDD

In [14]:
names.collect()

['Adam',
 'Cray',
 'Shaun',
 'Brain',
 'Mark',
 'Christ',
 'Shail',
 'Satya',
 'Mark',
 'Norby',
 'Frans',
 'Mark',
 'Bill']

In [15]:
names.countByValue()

defaultdict(int,
            {'Adam': 1,
             'Bill': 1,
             'Brain': 1,
             'Christ': 1,
             'Cray': 1,
             'Frans': 1,
             'Mark': 3,
             'Norby': 1,
             'Satya': 1,
             'Shail': 1,
             'Shaun': 1})

In [16]:
def f(x): print(x)

a = sc.parallelize([1,2,3,4,5]).foreach(lambda x: print(x))

In [17]:
type(a)

NoneType

In [18]:
names.take(5)

['Adam', 'Cray', 'Shaun', 'Brain', 'Mark']

In [21]:
# employees = sc.textFile('employee.txt')

In [22]:
# type(employees)

pyspark.rdd.RDD

In [25]:
names.first()

'Adam'

In [26]:
names.count()

13

In [27]:
names.top(5)

['Shaun', 'Shail', 'Satya', 'Norby', 'Mark']

In [28]:
names.top(19)

['Shaun',
 'Shail',
 'Satya',
 'Norby',
 'Mark',
 'Mark',
 'Mark',
 'Frans',
 'Cray',
 'Christ',
 'Brain',
 'Bill',
 'Adam']

In [29]:
names.distinct().count()

11

In [44]:
num = sc.parallelize([5,5,4,3,2,9,2],2)
num.collect()

[5, 5, 4, 3, 2, 9, 2]

In [45]:
num.countByValue()

defaultdict(int, {2: 2, 3: 1, 4: 1, 5: 2, 9: 1})

In [46]:
type(num)

pyspark.rdd.RDD

## glom()

In [47]:
num.glom().collect()

[[5, 5, 4], [3, 2, 9, 2]]

In [48]:
type(num.glom())

pyspark.rdd.PipelinedRDD

In [51]:
num.take(4)

[5, 5, 4, 3]

In [49]:
num2 = sc.parallelize([5,5,4,3,2,9,2], 6)
num2.collect()

[5, 5, 4, 3, 2, 9, 2]

In [50]:
num2.glom().collect()

[[5], [5], [4], [3], [2], [9, 2]]

In [52]:
num2.glom().collect()[1:]

[[5], [4], [3], [2], [9, 2]]

In [53]:
num.max()

9

In [54]:
num.min()

2

In [55]:
num.mean()

4.285714285714286

## reduce

In [70]:
num.glom().collect()

[[5, 5, 4], [3, 2, 9, 2]]

In [57]:
num.reduce(lambda a,b:a+b)

30

In [58]:
num.reduce(lambda a,b:a*b)

10800

In [59]:
# get max
num.reduce(lambda x,y: x if x > y else y)

9

In [63]:
def myfun(a,b):
  print(a,b)
  print(a*2+b*2)
  return a*2 + b*2

In [64]:
num.reduce(myfun)

48 80
256


256

In [69]:
test = sc.parallelize([1,2],2)
test.reduce(myfun)

1 2
6


6

In [68]:
test2 = sc.parallelize([1,2,3],2)
test2.reduce(myfun)

1 10
22


22

In [65]:
num.takeOrdered(3)

[2, 2, 3]

## fold

In [80]:
test2.glom().collect()

[[1], [2, 3]]

In [81]:
# 1+1 = 2
# 1+2+3 = 7
# 2+7 = 9
test2.fold(1,lambda a,b: a+b)

9

In [82]:
# 1*1 = 1
# 1*2*3 = 6
# 1*6 = 6
test2.fold(1,lambda a,b: a*b)

6

In [83]:
# 2+1 = 3
# 2+2+3 = 7
# 2+3+7 = 12 
test2.fold(2,lambda a,b: a+b)

12

In [84]:
# 2*1 = 2
# 2*2*3 = 12
# 2*2*12 = 48
test2.fold(2,lambda a,b: a*b)

48

In [89]:
from operator import add,mul
b = sc.parallelize([1,2,3])
b.fold(1,add)

8

In [88]:
from operator import add,mul
b = sc.parallelize([1,2,3],2)
b.fold(1,add)

9

In [91]:
from operator import add,mul
b = sc.parallelize([1,2,3],2)
b.fold(1,mul)

6

In [92]:
c = sc.parallelize(range(1,10))

In [94]:
c.collect()

[1, 2, 3, 4, 5, 6, 7, 8, 9]